## This is a notebook that explores the file which is the result of the bash script, that merges trips and leavetimes with one another

In [1]:
import pandas as pd
import os, sys


#reading in data
file_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(file_dir)
newPath = os.path.join(parent_dir, 'bus_lines/bus_66_merge.csv')


df = pd.read_csv(newPath,  delimiter=',', index_col=False)


In [2]:
df.head(5)

,LineID,Direction,RouteID,DayOfService,TripID,ProgrNumber,StopPointID,PlannedTime_Arr,PlannedTime_Dep,ActualTime_Arr,ActualTime_Dep
0,66,1,66_11,2017-01-03,4084361,33,2218,39411,39411,38673,38673
1,66,1,66_11,2017-01-03,4084368,30,2216,69044,69044,68235,68235
2,66,1,66_11,2017-01-03,4084370,33,2218,32185,32185,31840,31840
3,66,1,66_11,2017-01-03,4084398,36,3366,80601,80601,80693,80693
4,66,1,66_11,2017-01-03,4084400,58,3918,44530,44530,44452,44452


In [3]:
file_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(file_dir)
newPath = os.path.join(parent_dir, '/home/student/data/weather/2017weatherClean.csv')


dfW17 = pd.read_csv(newPath, delimiter=',')

In [4]:
#Only keeping the columns that we want
dfW17  = dfW17[['dt','temp','pressure','humidity','wind_speed','weather_description','clouds_all','wind_deg']]

In [5]:
#We want to look at only one direction
df = df.loc[(df['Direction'] == 1)]

In [7]:
#combining DayOfService and TripID into one column 
df['comb'] = df['DayOfService'].map(str) + df['TripID'].map(str)
u = df['comb'].unique()

In [11]:
import numpy as np
trip_66 = df['TripID'].unique()
date_66 = df['DayOfService'].unique()

result = pd.DataFrame(columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration'])


# for trip in trip_46A:
#     for date in date_46A:
for date_trip in u:
        day_trip = df.loc[(df['comb'] == date_trip)].sort_values(by=['ProgrNumber'])
        day_trip.index = range(len(day_trip))

        # Calculate the time between each pair stops
        time = day_trip['ActualTime_Arr'].shift(-1) - day_trip['ActualTime_Arr']
        time.drop(time.tail(1).index,inplace=True)

        # Get the month and the day, month and the day of week
        month = pd.to_datetime(day_trip['DayOfService']).dt.month
        day = pd.to_datetime(day_trip['DayOfService']).dt.day
        dayofweek = pd.to_datetime(day_trip['DayOfService']).dt.dayofweek

        # Add datetime and actual_arrive time as current unix time
        datetime = pd.DatetimeIndex(day_trip['DayOfService']).astype(np.int64)/1000000000 + day_trip['ActualTime_Arr']
        datetime = pd.DataFrame(data={'unixtime':datetime})

        # Conver float to int, this is for the following merge operation
        datetime['unixtime'] = datetime['unixtime'].astype(np.int64)

        # Set End point 
        end_point = day_trip['StopPointID'].shift(-1)
        end_point.drop(end_point.tail(1).index,inplace=True)

        # Merge these columns
        a = pd.concat([datetime,dayofweek, month, day, day_trip['ActualTime_Arr'], day_trip['StopPointID'], end_point, time], axis = 1)
        # Change the name of columns
        a.columns = ['dt','dayofweek','month','day','arrive_time','start_point','end_point','duration']
        a.drop(a.tail(1).index,inplace=True)
        dfW17 = dfW17.sort_values(by=['dt'])
        a = a.sort_values('dt')
        
        # Merge two tables
        r = pd.merge_asof(a, dfW17, on = "dt",direction='nearest')

        result = pd.concat([r, result])
        
result.head(5)

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



,arrive_time,clouds_all,day,dayofweek,dt,duration,end_point,humidity,month,pressure,start_point,temp,weather_description,wind_deg,wind_speed
0,63893,75.0,30,4,1498844693,188.0,312.0,76.0,6,1014.0,317,286.72,broken clouds,320.0,8.0
1,64081,75.0,30,4,1498844881,198.0,1444.0,76.0,6,1014.0,312,286.72,broken clouds,320.0,8.0
2,64279,75.0,30,4,1498845079,123.0,1445.0,76.0,6,1014.0,1444,286.72,broken clouds,320.0,8.0
3,64402,75.0,30,4,1498845202,232.0,7078.0,76.0,6,1014.0,1445,286.72,broken clouds,320.0,8.0
4,64634,75.0,30,4,1498845434,130.0,1449.0,76.0,6,1014.0,7078,286.72,broken clouds,320.0,8.0
